In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import os

import vdmlab as vdm

from loading_data import get_data
from utils_maze import spikes_by_position
from utils_plotting import plot_intersects, plot_zone
from startup import load_shortcut_position

In [ ]:
import info.r068d8 as info

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)

In [ ]:
u_line = LineString(info.u_trajectory)
shortcut_line = LineString(info.shortcut_trajectory)
novel_line = LineString(info.novel_trajectory)

expand_by = 6

u_start = Point(info.u_trajectory[0])
u_stop = Point(info.u_trajectory[-1])
shortcut_start = Point(info.shortcut_trajectory[0])
shortcut_stop = Point(info.shortcut_trajectory[-1])
novel_start = Point(info.novel_trajectory[0])
novel_stop = Point(info.novel_trajectory[-1])
pedestal_center = Point(info.path_pts['pedestal'][0], info.path_pts['pedestal'][1])
pedestal = pedestal_center.buffer(expand_by*2.2)

In [ ]:
zone = dict()
zone['u'] = vdm.expand_line(u_start, u_stop, u_line, expand_by)
zone['shortcut'] = vdm.expand_line(shortcut_start, shortcut_stop, shortcut_line, expand_by)
zone['novel'] = vdm.expand_line(novel_start, novel_stop, novel_line, expand_by)
zone['ushort'] = zone['u'].intersection(zone['shortcut'])
zone['unovel'] = zone['u'].intersection(zone['novel'])
zone['uped'] = zone['u'].intersection(pedestal)
zone['shortped'] = zone['shortcut'].intersection(pedestal)
zone['novelped'] = zone['novel'].intersection(pedestal)
zone['pedestal'] = pedestal

In [ ]:
plt.plot(position.x, position.y, 'y.', ms=1)
plt.plot(u_line.xy[0], u_line.xy[1], 'k.')
plt.plot(shortcut_line.xy[0], shortcut_line.xy[1], 'r.')
plt.plot(novel_line.xy[0], novel_line.xy[1], 'g.')
for zones in zone:
    if zone[zones].geom_type == 'MultiPolygon':
        plot_intersects(zone[zones])
    elif zone[zones].geom_type == 'Polygon':
        plot_zone(zone[zones])
    else:
        continue

In [ ]:
u_idx = []
shortcut_idx = []
novel_idx = []
ped_idx = []
other_idx = []
for i, (x, y) in enumerate(zip(position.x, position.y)):
    point = Point([x, y])
    if pedestal.contains(point) or zone['uped'].contains(point) or zone['shortped'].contains(point) or zone['novelped'].contains(point):
        ped_idx.append(i)
        continue
    elif zone['u'].contains(point) or zone['ushort'].contains(point) or zone['unovel'].contains(point):
        u_idx.append(i)
        continue
    elif zone['shortcut'].contains(point):
        shortcut_idx.append(i)
        continue
    elif zone['novel'].contains(point):
        novel_idx.append(i)
        continue
    else:
        other_idx.append(i)

In [ ]:
u_position = position[u_idx]
shortcut_position = position[shortcut_idx]
novel_position = position[novel_idx]
pedestal_position = position[ped_idx]
other_position = position[other_idx]

plt.plot(u_position.x, u_position.y, 'b.')
plt.plot(shortcut_position.x, shortcut_position.y, 'g.')
plt.plot(novel_position.x, novel_position.y, 'r.')
plt.plot(pedestal_position.x, pedestal_position.y, 'm.')
plt.plot(other_position.x, other_position.y, 'k.')
plt.show()